In [1]:
#Python
from datetime import datetime

#Models
from models.oven_db import Oven
from models.nut_batch_db import NutBatch, NutType
from models.real_butch_data_db import RealTimeBatchData

#Services
from services.oven_service import OvenService
from services.nut_batch_service import NutBatchService
from services.real_batch_service import RealTimeBatchDataService

In [2]:
oven_id = "67f3f1d0f93c97b2aec11525"
oven_service = OvenService()
nut_batch_service = NutBatchService()
real_batch_service = RealTimeBatchDataService()

In [3]:
#Crear un horno
oven = Oven("Horno 3", "Chilecito")
oven_service.create_oven(oven)

In [4]:
#Obtener todos los hornos disponibles
result = oven_service.list_ovens()
print(result)

[{'id': '67f3f1d0f93c97b2aec11525', 'name': 'Horno de prueba'}, {'id': '67f3f0fff93c97b2aec11524', 'name': 'Horno de eliminacion'}, {'id': '68096ffadde6e01814b090d7', 'name': 'Horno 3'}]


In [5]:
#Obtener un horno
result = oven_service.get_oven("67f3f0fff93c97b2aec11524")
print(result)

{'_id': ObjectId('67f3f0fff93c97b2aec11524'), 'name': 'Horno de eliminacion', 'location': 'Guanchin'}


In [6]:
#Eliminar un horno
oven_service.delete_oven("67f3f0fff93c97b2aec11524")

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

In [7]:
#Crear un lote de nueces previo al secado
nut_batch = NutBatch(
    nut_type=NutType.HALF,
    initial_weight=50.36,
    initial_humidity=36.25,
    target_humidity=20.23,
    oven_id="67f3f1d0f93c97b2aec11525"
)
result = nut_batch_service.create_batch(nut_batch)

In [8]:
#Obtener todos los lotes
batchs = nut_batch_service.list_batches()


for batch in batchs:
    print(f"""
Lote ID: {str(batch['_id'])}
Tipo de nuez: {batch['nut_type']}
Horno ID: {batch['oven_id']}
Estado: {batch['status']}

Fecha de inicio: {batch['start_date'].strftime('%Y-%m-%d %H:%M:%S')}
Fecha de fin: {batch['end_date'].strftime('%Y-%m-%d %H:%M:%S') if batch['end_date'] else '—'}

Peso inicial: {batch['initial_weight']} kg
Humedad inicial: {batch['initial_humidity']}%
Humedad objetivo: {batch['target_humidity']}%

Historial de secado: {len(batch.get('drying_history', []))} registros
""")


Lote ID: 68097005dde6e01814b090d8
Tipo de nuez: HALF
Horno ID: 67f3f1d0f93c97b2aec11525
Estado: IN_PROGRESS

Fecha de inicio: 2025-04-23 19:56:05
Fecha de fin: —

Peso inicial: 50.36 kg
Humedad inicial: 36.25%
Humedad objetivo: 20.23%

Historial de secado: 0 registros



In [9]:
#Insertar datos temporales en redis sobre los lotes
batch = nut_batch_service.get_actual_batch_by_oven(oven_id)
data = RealTimeBatchData(
    batch_id=str(batch["_id"]),
    temperature=12,
    humidity=17,
    weight=10,
    gas_consumption=18,
)

real_batch_service.update_data(data)



In [10]:
#Obtener todos los registros temporales para un lote



data = real_batch_service.get_all_data("67f51e064126a6e6cba17bc1")
for i, d in enumerate(data, start=1):
    print(f"Registro #{i} Lote: {d['batch_id']} Temperatura: {d['temperature']}°C Humedad: {d['humidity']}% Peso: {d['weight']}kg Gas consumido: {d['gas_consumption']} LTimestamp: {datetime.fromisoformat(d['timestamp']).strftime('%Y-%m-%d %H:%M:%S')}")

Registro #1 Lote: 67f51e064126a6e6cba17bc1 Temperatura: 55°C Humedad: 31% Peso: 13kg Gas consumido: 12 LTimestamp: 2025-04-08 23:20:04
Registro #2 Lote: 67f51e064126a6e6cba17bc1 Temperatura: 53°C Humedad: 30% Peso: 12kg Gas consumido: 15 LTimestamp: 2025-04-08 23:20:29
Registro #3 Lote: 67f51e064126a6e6cba17bc1 Temperatura: 12°C Humedad: 17% Peso: 10kg Gas consumido: 18 LTimestamp: 2025-04-08 23:27:17
Registro #4 Lote: 67f51e064126a6e6cba17bc1 Temperatura: 12°C Humedad: 17% Peso: 10kg Gas consumido: 18 LTimestamp: 2025-04-08 23:27:45
Registro #5 Lote: 67f51e064126a6e6cba17bc1 Temperatura: 12°C Humedad: 17% Peso: 10kg Gas consumido: 18 LTimestamp: 2025-04-09 00:06:34
Registro #6 Lote: 67f51e064126a6e6cba17bc1 Temperatura: 12°C Humedad: 17% Peso: 10kg Gas consumido: 18 LTimestamp: 2025-04-09 00:08:16


In [ ]:
#Finalizacion de proceso de secado
#Obtiene todos los registros de Redis correspondiente a ese lote de nueces y si supera los targes especificados en la creacion este coloca el lote como finalizado
result = nut_batch_service.add_drying_data(oven_id)
print(result)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)
